# Membuat Alur

Anda dapat melakukan berbagai langkah yang diperlukan untuk menyerap data, melatih model, dan mendaftarkan model satu per satu dengan menggunakan SDK Azure Machine Learning untuk menjalankan eksperimen berbasis skrip. Tetapi, dalam lingkungan perusahaan, merupakan hal yang umum untuk menyertakan urutan langkah-langkah terpisah yang diperlukan untuk membangun solusi pembelajaran mesin ke dalam *alur* yang dapat dijalankan pada satu atau beberapa target komputasi; baik sesuai permintaan pengguna, dari proses pembuatan otomatis, atau sesuai jadwal.

Dalam buku catatan ini, Anda akan menyatukan semua elemen ini untuk membuat alur sederhana yang memproses data sebelumnya, lalu melatih dan mendaftarkan model.

## Menghubungkan ke ruang kerja Anda

Untuk memulai, hubungkan ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Mempersiapkan data

Di alur, Anda akan menggunakan himpunan data yang berisi detail pasien diabetes. Jalankan sel di bawah ini untuk membuat himpunan data ini (jika Anda membuat himpunan data sebelumnya, kode akan menemukan versi yang ada)

In [ ]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    Dataset.File.upload_directory(src_dir='data',
                              target=DataPath(default_ds, 'diabetes-data/')
                              )

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## Membuat skrip untuk langkah-langkah alur

Alur terdiri dari satu atau lebih *langkah*, yang dapat berupa skrip Python, atau langkah khusus seperti langkah transfer data yang menyalin data dari satu lokasi ke lokasi lain. Setiap langkah dapat berjalan dalam konteks komputasi langkah tersendiri. Dalam latihan ini, Anda akan membuat alur sederhana yang berisi dua langkah skrip Python: satu untuk melakukan pra-proses beberapa data pelatihan, dan satu lagi untuk menggunakan data yang telah diproses sebelumnya untuk melatih dan mendaftarkan model.

Pertama, mari kita buat folder untuk file skrip yang akan kita gunakan dalam langkah-langkah alur.

In [ ]:
import os
# Create a folder for the pipeline step files
experiment_folder = 'diabetes_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

Sekarang mari kita buat skrip pertama, yang akan membaca data dari himpunan data diabetes dan menerapkan beberapa pra-pemrosesan sederhana untuk menghapus setiap baris dengan data yang hilang dan menormalkan fitur numerik sehingga berada pada skala yang sama.

Skrip menyertakan argumen bernama **--prepped-data**, yang merujuk ke folder tempat data yang dihasilkan akan disimpan.

In [ ]:
%%writefile $experiment_folder/prep_diabetes.py
# Import libraries
import os
import argparse
import pandas as pd
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
parser.add_argument('--prepped-data', type=str, dest='prepped_data', default='prepped_data', help='Folder for results')
args = parser.parse_args()
save_folder = args.prepped_data

# Get the experiment run context
run = Run.get_context()

# load the data (passed as an input dataset)
print("Loading Data...")
diabetes = run.input_datasets['raw_data'].to_pandas_dataframe()

# Log raw row count
row_count = (len(diabetes))
run.log('raw_rows', row_count)

# remove nulls
diabetes = diabetes.dropna()

# Normalize the numeric columns
scaler = MinMaxScaler()
num_cols = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree']
diabetes[num_cols] = scaler.fit_transform(diabetes[num_cols])

# Log processed rows
row_count = (len(diabetes))
run.log('processed_rows', row_count)

# Save the prepped data
print("Saving Data...")
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder,'data.csv')
diabetes.to_csv(save_path, index=False, header=True)

# End the run
run.complete()

Sekarang Anda dapat membuat skrip untuk langkah kedua, yang akan melatih model. Skrip menyertakan argumen bernama **--training-data**, yang merujuk ke lokasi penyimpanan data yang disimpan oleh langkah sebelumnya.

In [ ]:
%%writefile $experiment_folder/train_diabetes.py
# Import libraries
from azureml.core import Run, Model
import argparse
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--training-data", type=str, dest='training_data', help='training data')
args = parser.parse_args()
training_data = args.training_data

# Get the experiment run context
run = Run.get_context()

# load the prepared data file in the training folder
print("Loading Data...")
file_path = os.path.join(training_data,'data.csv')
diabetes = pd.read_csv(file_path)

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train adecision tree model
print('Training a decision tree model...')
model = DecisionTreeClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

# Save the trained model in the outputs folder
print("Saving model...")
os.makedirs('outputs', exist_ok=True)
model_file = os.path.join('outputs', 'diabetes_model.pkl')
joblib.dump(value=model, filename=model_file)

# Register the model
print('Registering model...')
Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'diabetes_model',
               tags={'Training context':'Pipeline'},
               properties={'AUC': np.float(auc), 'Accuracy': np.float(acc)})


run.complete()

## Mempersiapkan lingkungan komputasi untuk langkah-langkah alur

Dalam latihan ini, Anda akan menggunakan komputasi yang sama untuk kedua langkah, tetapi penting untuk menyadari bahwa setiap langkah dijalankan secara independen; sehingga Anda dapat menentukan konteks komputasi yang berbeda untuk setiap langkah jika sesuai.

Pertama, dapatkan target komputasi yang Anda buat di lab sebelumnya (jika tidak ada, lab akan dibuat).

> **Penting**: Ubah *your-compute-cluster* menjadi nama kluster komputasi Anda dalam kode di bawah ini sebelum menjalankannya! Nama kluster harus berupa nama unik global dengan panjang antara 2 hingga 16 karakter. Karakter yang valid adalah huruf, angka, dan - karakter.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

> **Catatan**: Instans dan kluster komputasi didasarkan pada gambar mesin virtual Azure standar. Untuk latihan ini, gambar *Standard_DS11_v2* direkomendasikan untuk mencapai keseimbangan optimal antara biaya dan performa. Jika langganan Anda memiliki kuota yang tidak menyertakan gambar ini, pilih gambar alternatif; tetapi perlu diingat bahwa gambar yang lebih besar mungkin memerlukan biaya lebih tinggi dan gambar yang lebih kecil mungkin tidak cukup untuk menyelesaikan tugas. Atau, minta administrator Azure Anda untuk memperbesar kuota Anda.

Komputasi akan membutuhkan lingkungan Python dengan dependensi paket yang diperlukan diinstal.

In [ ]:
%%writefile $experiment_folder/experiment_env.yml
name: experiment_env
dependencies:
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Sekarang setelah Anda memiliki file konfigurasi Conda, Anda dapat membuat lingkungan dan menggunakannya dalam konfigurasi eksekusi untuk alur.

In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/experiment_env.yml")

# Register the environment 
experiment_env.register(workspace=ws)
registered_env = Environment.get(ws, 'experiment_env')

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

## Membuat dan menjalankan alur

Sekarang Anda siap untuk membuat dan menjalankan alur.

Pertama, Anda perlu menentukan langkah-langkah untuk alur, dan referensi data apa pun yang perlu diteruskan di antara keduanya. Dalam kasus ini, langkah pertama harus menulis data yang telah disiapkan ke folder yang dapat dibaca oleh langkah kedua. Karena langkah-langkah tersebut akan dijalankan pada komputasi jarak jauh (dan kenyataannya, masing-masing dapat dijalankan pada komputasi yang berbeda), jalur folder harus diteruskan sebagai referensi data ke lokasi di penyimpanan data dalam ruang kerja. Objek **OutputFileDatasetConfig** adalah jenis referensi data khusus yang digunakan untuk lokasi penyimpanan sementara yang dapat diteruskan di antara langkah-langkah alur, jadi Anda akan membuatnya dan menggunakan at sebagai output untuk langkah pertama dan input untuk tahap kedua. Perhatikan bahwa Anda harus meneruskannya sebagai argumen skrip sehingga kode Anda dapat mengakses lokasi penyimpanan data yang dirujuk oleh referensi data.

In [ ]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
prepped_data = OutputFileDatasetConfig("prepped_data")

# Step 1, Run the data prep script
prep_step = PythonScriptStep(name = "Prepare Data",
                                source_directory = experiment_folder,
                                script_name = "prep_diabetes.py",
                                arguments = ['--input-data', diabetes_ds.as_named_input('raw_data'),
                                             '--prepped-data', prepped_data],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
train_step = PythonScriptStep(name = "Train and Register Model",
                                source_directory = experiment_folder,
                                script_name = "train_diabetes.py",
                                arguments = ['--training-data', prepped_data.as_input()],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

OK, Anda sudah siap membangun alur dari langkah-langkah yang telah Anda tetapkan dan menjalankannya sebagai eksperimen.

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [prep_step, train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'mslearn-diabetes-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

Representasi grafis dari eksperimen alur akan ditampilkan di widget saat dijalankan. Perhatikan indikator kernel di kanan atas halaman, ketika indikator berubah dari **&#9899;** menjadi **&#9711;**, kode telah selesai dijalankan. Anda juga dapat memantau eksekusi alur di halaman **Eksperimen** di [Studio Azure Machine Learning](https://ml.azure.com).

Saat alur selesai, Anda dapat memeriksa metrik yang dicatat oleh eksekusi turunan alur.

In [ ]:
for run in pipeline_run.get_children():
    print(run.name, ':')
    metrics = run.get_metrics()
    for metric_name in metrics:
        print('\t',metric_name, ":", metrics[metric_name])

Dengan asumsi alur berhasil, model baru akan didaftarkan dengan tag *Konteks pelatihan* yang menunjukkan bahwa model tersebut dilatih dalam alur. Jalankan kode berikut untuk memverifikasi hal ini.

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## Menerbitkan alur

Setelah membuat dan menguji alur, Anda dapat menerbitkan alur sebagai layanan REST.

In [ ]:
# Publish the pipeline from the run
published_pipeline = pipeline_run.publish_pipeline(
    name="diabetes-training-pipeline", description="Trains diabetes model", version="1.0")

published_pipeline

Perhatikan bahwa alur yang diterbitkan memiliki titik akhir, yang dapat Anda lihat di halaman **Titik akhir** (pada tab **Titik Akhir Alur**) di [Studio Azure Machine Learning](https://ml.azure.com). Anda juga dapat menemukan URI alur sebagai properti dari objek alur yang diterbitkan:

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

## Memanggil titik akhir alur

Untuk menggunakan titik akhir, aplikasi klien perlu melakukan panggilan REST melalui HTTP. Permintaan ini harus diautentikasi, sehingga header otorisasi diperlukan. Aplikasi yang sebenarnya akan memerlukan perwakilan layanan yang dapat digunakan untuk diautentikasi, tetapi untuk mengujinya, kita akan menggunakan header otorisasi dari koneksi Anda saat ini ke ruang kerja Azure Anda, yang dapat Anda peroleh menggunakan kode berikut:

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print("Authentication header ready.")

Sekarang kita siap untuk memanggil antarmuka REST. Alur berjalan secara asinkron, jadi kita akan mendapatkan pengidentifikasi kembali, yang dapat kita gunakan untuk melacak eksperimen alur saat dijalankan:

In [ ]:
import requests

experiment_name = 'mslearn-diabetes-pipeline'

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

Karena Anda memiliki ID eksekusi, Anda dapat menggunakannya untuk menunggu eksekusi selesai.

> **Catatan**: Alur akan selesai dengan cepat, karena setiap langkah dikonfigurasi untuk memungkinkan penggunaan kembali output. Hal ini dilakukan terutama untuk kenyamanan dan untuk menghemat waktu dalam kursus ini. Pada kenyataannya, Anda mungkin ingin langkah pertama dijalankan setiap kali data berubah, dan memicu langkah-langkah berikutnya hanya jika output dari langkah pertama berubah.

In [ ]:
from azureml.pipeline.core.run import PipelineRun

published_pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
published_pipeline_run.wait_for_completion(show_output=True)

## Menjadwalkan Alur

Misalkan klinik untuk pasien diabetes mengumpulkan data baru setiap minggu, dan menambahkannya ke himpunan data. Anda dapat menjalankan alur setiap minggu untuk melatih ulang model dengan data baru.

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Submit the Pipeline every Monday at 00:00 UTC
recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")
weekly_schedule = Schedule.create(ws, name="weekly-diabetes-training", 
                                  description="Based on time",
                                  pipeline_id=published_pipeline.id, 
                                  experiment_name='mslearn-diabetes-pipeline', 
                                  recurrence=recurrence)
print('Pipeline scheduled.')

Anda dapat mengambil jadwal yang ditentukan di ruang kerja seperti ini:

In [ ]:
schedules = Schedule.list(ws)
schedules

Anda dapat memeriksa eksekusi terbaru seperti ini:

In [ ]:
pipeline_experiment = ws.experiments.get('mslearn-diabetes-pipeline')
latest_run = list(pipeline_experiment.get_runs())[0]

latest_run.get_details()

Ini adalah contoh sederhana, yang dirancang untuk menunjukkan prinsip. Pada kenyataannya, Anda dapat membangun logika yang lebih canggih ke dalam langkah-langkah alur - misalnya, mengevaluasi model terhadap beberapa data pengujian untuk menghitung metrik performa seperti AUC atau akurasi, membandingkan metrik dengan versi model yang terdaftar sebelumnya, dan hanya mendaftarkan model baru jika performa model lebih baik.

Anda dapat menggunakan [ekstensi Azure Machine Learning untuk Azure DevOps](https://marketplace.visualstudio.com/items?itemName=ms-air-aiagility.vss-services-azureml) untuk menggabungkan alur Azure Machine Learning dengan alur Azure DevOps (ya, *memang* membingungkan bahwa keduanya memiliki nama yang sama!) dan mengintegrasikan pelatihan ulang model ke dalam proses *integrasi berkelanjutan/penyebaran berkelanjutan (CI/CD)*. Misalnya, Anda dapat menggunakan alur *pembangunan* Azure DevOps untuk memicu alur Azure Machine Learning yang melatih dan mendaftarkan model, dan ketika model didaftarkan, model dapat memicu alur *rilis* Azure Devops yang menyebarkan model sebagai layanan web, bersama dengan aplikasi atau layanan yang menggunakan model.